# Notebook to show workflow

### plots stuff

In [1]:
import matplotlib.pyplot as plt
from scipy.ndimage.filters import uniform_filter1d

plt.rcParams['figure.figsize'] = [15, 5]
EPS = 0.000001

### some tricks

In [2]:
import sys
import os

search_tools_path = os.path.join(os.getcwd(), 'search_tools')
if search_tools_path not in sys.path:
    sys.path.append(search_tools_path)

In [3]:
%load_ext autoreload
%autoreload 2

### any workflow needs these imports

In [4]:
from moving_ai.mai_map_reader import MaiReader, MaiMaps
from pipeline import *
from run_result import RunResult

# Let's test A*

In [5]:
from algorithms.astar import Astar
from algorithms.astar_heuristics import diagonal_dist

### Let's define processor which checks that A* is correct

Processors expects a_result of type `List[Tuple[str, List[RunResult]]]`

(i.e. each element of list is $ (\mathrm{algorithm\_name}, [res_1, res_2, res_3,..]) $)

In [6]:
def process(a_results):
    for a_name, rs in a_results:
        for r in rs:
            if abs(r.act_len - r.opt_len) > EPS:
                print(f'Wrong result for algorithm: {a_name}')
                print(f'task: {r.task}, expected: {r.opt_len}, actual: {r.act_len}')
                return
    print('Everything is ok!')

### Let's build simple pipeline

In [7]:
def build_dummy_pipeline():
    return Pipeline(
        reader=MaiReader(MaiMaps.STARCRAFT),
        algorithms=[Astar(diagonal_dist)],
        res_builder=RunResult.create,
        processor=LProcessor(process)
    )

p = build_dummy_pipeline()
p.run()

KeyboardInterrupt: 

# Let's compare manhattan and diagonal

In [ ]:
from algorithms.astar_heuristics import diagonal_dist, manhattan_dist
from tabulate import tabulate

In [ ]:
smooth = lambda ys, n: uniform_filter1d(ys, n)
mean = lambda xs: sum(xs) / len(xs)

def plot_all_results(all_results, smooth_sz = 10, smooth_sz_2 = 10):
    suboptimals = {}
    fig, (ax1, ax2) = plt.subplots(1, 2)
    
    for h_name, rs in all_results:
        sorted_rs = sorted(rs, key=lambda r:r.opt_len)
        sorted_rs = [i for i in sorted_rs if i.opt_len != 0]
        xs = [i.opt_len for i in sorted_rs]
        ys1 = [i.nodes_created for i in sorted_rs]
        ax1.plot(xs, smooth(ys1, smooth_sz), label=h_name)
        ax1.title.set_text('[Number of expanded (Y)] on [length of optimal path (X)]')
        
        ys2 = [i.act_len / i.opt_len for i in sorted_rs]
        ax2.plot(xs, smooth(ys2, smooth_sz_2), label=h_name)
        ax2.title.set_text('[Actual / Optimal lengths (Y)] on [length of optimal path (X)]')
        
        subs = [i.act_div_opt() for i in sorted_rs if i.act_div_opt() > 1]
        if len(subs) > 0:
            suboptimals[h_name] = subs
    
    for ax in (ax1, ax2): ax.legend()
    plt.show()
        
    if len(suboptimals) > 0:
        print('Suboptimals:')
        si = suboptimals.items()
        header = [''] + [h.replace('diagonal', 'diag') for h, _ in si]
        cnt = ['count'] + [f'{len(subopt)}' for _, subopt in si]
        cnt_prc = ['count %'] + [f'{100 * len(subopt)/len(all_results[0][1]):.1f} %' for _, subopt in si]
        worst_ratio = ['worst ratio'] + [f'{max(subopt):.2f}' for _, subopt in si]
        mean_ = ['mean subopt'] + [f'{mean(subopt):.2f}' for _, subopt in si]
        print(tabulate([header, cnt, cnt_prc, worst_ratio, mean_]))

In [ ]:
p2 = Pipeline(
    reader = p.reader,
    algorithms = [Astar(diagonal_dist), Astar(manhattan_dist)],
    processor = plot_all_results
)

In [ ]:
p2.run()